In [1]:
# IMporting all the required libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

#mapping tools
!pip install geopy
from geopy.geocoders import Nominatim
!pip install folium
import folium

     |████████████████████████████████| 92kB 8.8MB/s eta 0:00:011


In [2]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,Rent per Unit,Lattitude,Longitude
0,Juhu,"95,184",19.11,72.83
1,Santacruz,"90,000",19.08,72.84
2,Bandra,"88,214",19.06,72.83
3,Powai,"85,000",19.12,72.91
4,Girgaon,"81,000",18.96,72.82


In [3]:
data.isnull().any() #Checking for NULL values in the dataset

Neighbourhood      False
 Rent per Unit     False
Lattitude          False
Longitude          False
dtype: bool

In [9]:
data.dtypes 

Neighbourhood       object
 Rent per Unit      object
Lattitude          float64
Longitude          float64
dtype: object

In [4]:
#Obtaining the geographical co-ordinates for Mumbai

address = 'Mumbai, India'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


The geograpical coordinate of Mumbai are 18.9387711, 72.8353355.


In [5]:
#Creating a map of Mumbai with Folium

mumbaimap = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(data['Lattitude'], data['Longitude'], data['Neighbourhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.Marker(
                        [lat, lng],
                        radius=10,
                        popup=label,
                        color='green',
                        fill=True,
                        fill_color='white',
                        icon=folium.Icon(icon='cloud')
                        #fill_opacity=0.7
                        ).add_to(mumbaimap)
mumbaimap

In [6]:
#Defining Foursquare credentials

CLIENT_ID = 'BXR5FTXI2GQEUL3EYHEB4JSKQSJ2O20M1CRCNO1A520MSON5' # your Foursquare ID
CLIENT_SECRET = 'SCOOZBZMMJMICHN21Y5Q3ZQI10RMTL5SCIEPMJLGA4P2A3HX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXR5FTXI2GQEUL3EYHEB4JSKQSJ2O20M1CRCNO1A520MSON5
CLIENT_SECRET:SCOOZBZMMJMICHN21Y5Q3ZQI10RMTL5SCIEPMJLGA4P2A3HX


In [7]:
#Getting a neighborhood name with Lat Lon values

neighborhood_latitude = data.loc[0, 'Lattitude'] # neighborhood latitude value
neighborhood_longitude = data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Juhu are 19.11, 72.83.


In [9]:
#Searching for venues using Foursquare within a radius of 500 meters

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 
 
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BXR5FTXI2GQEUL3EYHEB4JSKQSJ2O20M1CRCNO1A520MSON5&client_secret=SCOOZBZMMJMICHN21Y5Q3ZQI10RMTL5SCIEPMJLGA4P2A3HX&v=20180605&ll=19.11,72.83&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9b40059388d70024ab88b2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Vile Parle West',
  'headerFullLocation': 'Vile Parle West, Mumbai',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 22,
  'suggestedBounds': {'ne': {'lat': 19.114500004500005,
    'lng': 72.83475356282725},
   'sw': {'lat': 19.105499995499994, 'lng': 72.82524643717275}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50ba3776e4b062ae37022f54',
       'name': 'Facing East',
       'location': {'address': '13th Road, Opposite Lotus Eye Hospital',
        'crossStreet': 'Next to Natural Ice-Cream',
        'lat': 19.110120035881774,
        'lng': 72.8274

In [11]:
# function to extract the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Facing East,Chinese Restaurant,19.110120,72.827454
1,Copa,Cocktail Bar,19.109765,72.827172
2,B'wiched,American Restaurant,19.111257,72.826615
3,Fable,Pizza Place,19.111077,72.828135
4,Float bar,Lounge,19.108366,72.826863


In [13]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

22 venues were returned by Foursquare.


## Explore other Neighborhoods in Mumbai

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
mumbai_venues = getNearbyVenues(names=data['Neighbourhood'],
                                   latitudes=data['Lattitude'],
                                   longitudes=data['Longitude']
                                  )

Juhu
Santacruz
Bandra
Powai
Girgaon
Vile Parle
Andheri
Vikhroli
Chembur
Malad
Kandivali
Goregaon
Ghatkopar


In [16]:
#Filtering the venue categories by restaurants 

print(mumbai_venues.shape)

mumbai_venues = mumbai_venues[mumbai_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)

mumbai_venues

(284, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Juhu,19.11,72.83,Facing East,19.110120,72.827454,Chinese Restaurant
1,Juhu,19.11,72.83,B'wiched,19.111257,72.826615,American Restaurant
2,Juhu,19.11,72.83,noodle bar,19.110671,72.825431,Asian Restaurant
3,Juhu,19.11,72.83,Govinda's,19.113324,72.827022,Indian Restaurant
4,Santacruz,19.08,72.84,Shabari Restaurant,19.082411,72.840759,Indian Restaurant
5,Santacruz,19.08,72.84,Nice Fast Food Corner,19.077202,72.837742,Fast Food Restaurant
6,Santacruz,19.08,72.84,Gopal Krishna,19.080892,72.842424,Indian Restaurant
7,Santacruz,19.08,72.84,Fatkong Restaurant,19.078744,72.837806,Chinese Restaurant
8,Santacruz,19.08,72.84,Dynasty,19.078360,72.837656,Chinese Restaurant
9,Bandra,19.06,72.83,Pali Bhavan,19.062089,72.829459,Indian Restaurant


In [17]:
mumbai_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Andheri,7,7,7,7,7,7
Bandra,24,24,24,24,24,24
Chembur,7,7,7,7,7,7
Ghatkopar,6,6,6,6,6,6
Girgaon,4,4,4,4,4,4
Goregaon,4,4,4,4,4,4
Juhu,4,4,4,4,4,4
Kandivali,7,7,7,7,7,7
Malad,2,2,2,2,2,2


In [22]:
print('There are {} uniques categories in restaurants'.format(len(mumbai_venues['Venue Category'].unique())))

There are 16 uniques categories in restaurants


## Analyze Each Neighborhood

In [19]:
# one hot encoding
mumbai_onehot = pd.get_dummies(mumbai_venues[['Venue Category']], prefix="", prefix_sep="")

# add Neighbourhood column back to dataframe
mumbai_onehot['Neighbourhood'] = mumbai_venues['Neighborhood'] 

# move Neighbourhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

mumbai_onehot.head()

,Neighbourhood,American Restaurant,Asian Restaurant,Chinese Restaurant,Fast Food Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Maharashtrian Restaurant,North Indian Restaurant,Punjabi Restaurant,Restaurant,Seafood Restaurant,Spanish Restaurant,Sushi Restaurant,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant
0,Juhu,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Juhu,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Juhu,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Juhu,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,Santacruz,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [20]:
mumbai_onehot.shape

(102, 17)

In [21]:
mumbai_grouped = mumbai_onehot.groupby('Neighbourhood').mean().reset_index()
mumbai_grouped

,Neighbourhood,American Restaurant,Asian Restaurant,Chinese Restaurant,Fast Food Restaurant,French Restaurant,Indian Restaurant,Italian Restaurant,Maharashtrian Restaurant,North Indian Restaurant,Punjabi Restaurant,Restaurant,Seafood Restaurant,Spanish Restaurant,Sushi Restaurant,Tex-Mex Restaurant,Vegetarian / Vegan Restaurant
0,Andheri,0.00,0.000000,0.142857,0.000000,0.000000,0.571429,0.142857,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.142857
1,Bandra,0.00,0.125000,0.166667,0.041667,0.041667,0.416667,0.041667,0.000000,0.00,0.000000,0.041667,0.083333,0.041667,0.00,0.00,0.000000
2,Chembur,0.00,0.142857,0.142857,0.285714,0.000000,0.285714,0.000000,0.000000,0.00,0.142857,0.000000,0.000000,0.000000,0.00,0.00,0.000000
3,Ghatkopar,0.00,0.000000,0.000000,0.000000,0.000000,0.833333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.166667
4,Girgaon,0.00,0.000000,0.000000,0.250000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.250000
5,Goregaon,0.00,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
6,Juhu,0.25,0.250000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
7,Kandivali,0.00,0.000000,0.000000,0.142857,0.000000,0.857143,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
8,Malad,0.00,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000
9,Powai,0.00,0.040000,0.120000,0.040000,0.000000,0.400000,0.120000,0.000000,0.04,0.000000,0.120000,0.040000,0.000000,0.04,0.04,0.000000


In [96]:
mumbai_grouped.shape

(12, 17)

In [24]:
#Identifying the top 5 categories across all restaurants in the neighborhoods

num_top_venues = 5

for hood in mumbai_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = mumbai_grouped[mumbai_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Andheri----
                           venue  freq
0              Indian Restaurant  0.57
1             Chinese Restaurant  0.14
2             Italian Restaurant  0.14
3  Vegetarian / Vegan Restaurant  0.14
4            American Restaurant  0.00


----Bandra----
                  venue  freq
0     Indian Restaurant  0.42
1    Chinese Restaurant  0.17
2      Asian Restaurant  0.12
3    Seafood Restaurant  0.08
4  Fast Food Restaurant  0.04


----Chembur----
                  venue  freq
0  Fast Food Restaurant  0.29
1     Indian Restaurant  0.29
2      Asian Restaurant  0.14
3    Chinese Restaurant  0.14
4    Punjabi Restaurant  0.14


----Ghatkopar----
                           venue  freq
0              Indian Restaurant  0.83
1  Vegetarian / Vegan Restaurant  0.17
2            American Restaurant  0.00
3               Asian Restaurant  0.00
4             Chinese Restaurant  0.00


----Girgaon----
                           venue  freq
0              Indian Restaurant  0.50
1    

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
#Sorting the dataset in top 10 categories across all the neighborhoods

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighbourhoods_venues_sorted['Neighbourhood'] = mumbai_grouped['Neighbourhood']

for ind in np.arange(mumbai_grouped.shape[0]):
    Neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mumbai_grouped.iloc[ind, :], num_top_venues)

Neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Andheri,Indian Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Chinese Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant
1,Bandra,Indian Restaurant,Chinese Restaurant,Asian Restaurant,Seafood Restaurant,Spanish Restaurant,Restaurant,Italian Restaurant,French Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant
2,Chembur,Indian Restaurant,Fast Food Restaurant,Punjabi Restaurant,Chinese Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant
3,Ghatkopar,Indian Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant,North Indian Restaurant,Maharashtrian Restaurant
4,Girgaon,Indian Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant,North Indian Restaurant
5,Goregaon,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant,North Indian Restaurant
6,Juhu,Indian Restaurant,Chinese Restaurant,Asian Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant
7,Kandivali,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant,North Indian Restaurant
8,Malad,Indian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Spanish Restaurant,Seafood Restaurant,Restaurant,Punjabi Restaurant,North Indian Restaurant
9,Powai,Indian Restaurant,Restaurant,Italian Restaurant,Chinese Restaurant,Tex-Mex Restaurant,Sushi Restaurant,Seafood Restaurant,North Indian Restaurant,Fast Food Restaurant,Asian Restaurant
